The notebook generates utterance pairs from tagged utterances, where tags are {B-START, I-START, B-OTHER, I-OTHER, O}.

From the start of every conversation, i.e. utterances with B-START tag, an utterance-reponse pair is construct by assuming the next utterance in the conversation is a reponse. We stop the pairing if we reach the end of the conversation.

e.g.


# Import Libraries and Data

In [1]:
import pandas as pd
import os
import datetime
pd.set_option('display.max_colwidth', -1)
pd.set_option("display.max_rows", 1000)

In [2]:
df_to_generate = pd.read_csv('../data/predicted/heuristic_pred.csv', index_col=[0])

In [3]:
def add_chapters(df):
    chapter_dict = dict()
    chapter_tag = ''
    for i in df.index:
        curr_chapter_tag = df.loc[i]['chapter_tag']
        if curr_chapter_tag == '':
            if chapter_tag != '':
                chapter_dict[i] = chapter_tag
            else:
                chapter_tag = curr_chapter_tag
                chapter_dict[i] = chapter_tag
        else:
            if chapter_tag == curr_chapter_tag:
                chapter_dict[i] = chapter_tag
            else:
                chapter_tag = curr_chapter_tag
                chapter_dict[i] = chapter_tag
    return chapter_dict

In [4]:
df_to_generate['chapter_tag'] = df_to_generate['sent'].apply(lambda x: x if 'chapter ' in x.lower() else '')
df_to_generate['chapter_tag'] = list(x[1] for x in add_chapters(df_to_generate).items())

# Generate Utterance Pairs

In [5]:
df_utter = df_to_generate[df_to_generate['label']!='O']
utter_list = list(df_utter.apply(lambda row: row['sent'], axis=1).values)

print('Number of utterances: {}'.format(len(utter_list)))

utter_pair_list = []
for idx, i in enumerate(utter_list):
    if idx < (len(utter_list)-1):
        utter_pair_list.append((i,)+(utter_list[idx+1],))
utter_pair_list.append(('',)+('',))

df_pairs = pd.DataFrame()
df_pairs['utter_1'] = [x[0] for x in utter_pair_list]
df_pairs['utter_2'] = [x[1] for x in utter_pair_list]
df_pairs['label'] = df_utter['label'].values
df_pairs['chapter_tag'] = df_utter['chapter_tag'].values

indices_to_rm = []
for i in df_pairs.index:
    if (df_pairs.loc[i]['label'] == 'B-START') & (i!=0):
        indices_to_rm.append(i-1)

df_pairs = df_pairs.drop(indices_to_rm, axis=0)
df_pairs = df_pairs.reset_index()
del df_pairs['index']

test_set_chapters = ['Chapter {}'.format(x) for x in range(19,27)]
validation_set_chapters = ['Chapter {}'.format(x) for x in range(27,34)]
def custom_train_test_split(field):
    if field in test_set_chapters:
        return 'test'
    elif field in validation_set_chapters:
        return 'validation'
    else:
        return 'train'
df_pairs['split_tag'] = df_pairs['chapter_tag'].apply(lambda x: custom_train_test_split(x))

print('Generated {} utterance pairs'.format(df_pairs.shape[0]))

print('Saving to csv..')

# Save as csv
NAME = 'heur'
dirname = os.path.dirname('__file__')
output_path = os.path.join(dirname, '../data/utterance_pairs/')
current_year = str(datetime.datetime.now())[0:10]
csv_name = '{}-utter-pairs-{}.csv'.format(NAME, current_year)
df_pairs.to_csv(output_path + csv_name)

print('Done')

Number of utterances: 1740
Generated 1648 utterance pairs
Saving to csv..
Done


# Preview

In [6]:
# preview
df_pairs_heuristic = pd.read_csv('../data/utterance_pairs/heur-utter-pairs-2019-05-06.csv', index_col=[0])

In [7]:
df_pairs_heuristic.head(30)

,utter_1,utter_2,label,chapter_tag,split_tag
0,"""My dear Mr Bennet,""","""have you heard that Netherfield Park is let at last?""",B-START,Chapter 1,train
1,"""have you heard that Netherfield Park is let at last?""","""But it is,""",I-START,Chapter 1,train
2,"""But it is,""","""for Mrs Long has just been here, and she told me all about it.""",B-OTHER,Chapter 1,train
3,"""for Mrs Long has just been here, and she told me all about it.""","""Do you not want to know who has taken it?""",I-OTHER,Chapter 1,train
4,"""Do you not want to know who has taken it?""","""You want to tell me, and I have no objection to hearing it.""",B-OTHER,Chapter 1,train
5,"""You want to tell me, and I have no objection to hearing it.""","""Why, my dear, you must know, Mrs Long says that Netherfield is taken by a young man of large fortune from the north of England; that he came down on Monday in a chaise and four to see the place, and was so much delighted with it, that he agreed with Mr Morris immediately; that he is to take possession before Michaelmas, and some of his servants are to be in the house by the end of next week.""",B-OTHER,Chapter 1,train
6,"""Why, my dear, you must know, Mrs Long says that Netherfield is taken by a young man of large fortune from the north of England; that he came down on Monday in a chaise and four to see the place, and was so much delighted with it, that he agreed with Mr Morris immediately; that he is to take possession before Michaelmas, and some of his servants are to be in the house by the end of next week.""","""What is his name?""",B-OTHER,Chapter 1,train
7,"""What is his name?""","""Bingley.""",B-OTHER,Chapter 1,train
8,"""Bingley.""","""Is he married or single?""",B-OTHER,Chapter 1,train
9,"""Is he married or single?""","""Oh! Single, my dear, to be sure! A single man of large fortune; four or five thousand a year. What a fine thing for our girls!""",B-OTHER,Chapter 1,train
